In [1]:
%matplotlib inline

import pandas;
import cv2;
import pickle;
import numpy;
import os;
import os.path;

from matplotlib import pyplot;

import tensorflow as tf;

In [2]:
images_dir = "/Users/ravishchawla/workspace/machinelearning/datasets/dogs_vs_cats/";
train_dir = images_dir + "train/";
test_dir = images_dir + "test/";

In [3]:
file_names_p = 'file_names.p';

train_file_names = [];
test_file_names = [];

if os.path.isfile(file_names_p):
    train_file_names, test_file_names = pickle.load(open(file_names_p, 'rb'));
    train_file_names = train_file_names[0];
    test_file_names = test_file_names[0];
    print('dataset loaded');
else:
    for root, folder, file in os.walk(train_dir):
        train_file_names.append(file);

    for root, folder, file in os.walk(test_dir):
        test_file_names.append(file);

    pickle.dump((train_file_names, test_file_names), open(file_names_p, 'wb'));

dataset loaded


In [9]:
max_size = [500, 500];

def load_images_and_labels(files_dir, image_names, size_sample):    
    image_set = numpy.random.choice(image_names, size=size_sample);
    
    loaded_images = [];
    labels_list = [];
    
    for fname in image_set:
        img = cv2.imread(files_dir + fname);
        
        if 'cat' in fname:
            labels_list.append([0,1]);
        elif 'dog' in fname:
            labels_list.append([1,0]);
        else:
            labels_list.append([0,0]);

        loaded_images.append(img);
        
    return [loaded_images, labels_list];

def process_images(images, border_size):
    images_list = [];

    print(len(images))
    for image_pos in range(0, len(images)):
        img = images[image_pos];
        
        width_diff = (border_size[1] - img.shape[1]);
        height_diff = (border_size[0] - img.shape[0]);
    
        width_diffs = [width_diff // 2, width_diff - (width_diff // 2)];
        height_diffs = [height_diff // 2, height_diff - (height_diff // 2)];
        
        bet_img = cv2.copyMakeBorder(img, height_diffs[0], height_diffs[1], width_diffs[0], width_diffs[1], cv2.BORDER_CONSTANT, value=[0,0,0]);
        
        bw_img = cv2.cvtColor(bet_img, cv2.COLOR_BGR2GRAY);
        
        flat_image = numpy.reshape(bw_img, [bw_img.shape[0] * bw_img.shape[1]]);
        #flat_image = flat_image.astype('float32');
        
        images_list.append(flat_image);
        
    
    images_list = numpy.array(images_list);
    
    return images_list;
#print(images_with_labels);

#Parameters

learning_rate = 0.001;
training_iters = 20000;
batch_size = 128;
display_step = 10;

num_train_images = 250;
num_test_images = 250;

train_and_test_set = numpy.random.choice(train_file_names, num_train_images + num_test_images);

train_images, train_labels = load_images_and_labels(train_dir, train_and_test_set[:num_train_images], num_train_images);
test_images, test_labels = load_images_and_labels(train_dir, train_and_test_set[num_train_images:num_train_images+num_test_images], num_test_images);

train_images = process_images(train_images, max_size);
test_images = process_images(test_images, max_size);

n_input = max_size[0] * max_size[1]; #data input
n_classes = 2; #total classes = 10
dropout = 0.75; #probability to keep units

x = tf.placeholder(tf.float32, [None, n_input]);
y = tf.placeholder(tf.float32, [None, n_classes]);
keep_prob = tf.placeholder(tf.float32) #dropout

print(max_size);

#train_images, train_labels = process_images_and_labels(train_dir, train_file_names, 5);
#test_images, test_labels = process_images_and_labels(test_dir, test_file_names, 250);

250
250
[500, 500]


In [83]:
filter_size = {
    'wc1' : 11,
    'wc2' : 5,
    'wc3' : 3
}

input_size = {
    'wc1' : 1,
    'wc2' : 96,
    'wc3' : 256,
    'fc1' : 384,
    'out' : 4096,
}

input_wh = {
    'wc1' : 500,
    'wc2' : 84,
    'wc3' : 42,
    'fc1' : 21,
}

num_filters = {
    'wc1' : 96,
    'wc2' : 256,
    'wc3' : 384,
    'fc1' : 4096
}

weights = {
    #5x5 convolution, 1 depth layer input, 96 depth layer outputs
    'wc1' : tf.Variable(tf.random_normal([filter_size['wc1'], filter_size['wc1'], input_size['wc1'], num_filters['wc1']])),
    
    'mp1' : 2,
    
    'wc2' : tf.Variable(tf.random_normal([filter_size['wc2'], filter_size['wc2'], input_size['wc2'], num_filters['wc2']])),
    
    'mp2' : 2,
    
    'wc3' : tf.Variable(tf.random_normal([filter_size['wc3'], filter_size['wc3'], input_size['wc3'], num_filters['wc3']])),
    
    'mp3' : 2,
    
    'fc1' : tf.Variable(tf.random_normal([input_wh['fc1'] * input_wh['fc1'] * input_size['fc1'], num_filters['fc1']])),
    
    'out' : tf.Variable(tf.random_normal([input_size['out'], n_classes]))
}

strides = {
    
    'wc1' : 3,
    'wc2' : 1,
    'wc3' : 1,
}

biases = {
    
    'bc1' : tf.Variable(tf.random_normal([num_filters['wc1']])),
    'bc2' : tf.Variable(tf.random_normal([num_filters['wc2']])),
    'bc3' : tf.Variable(tf.random_normal([num_filters['wc3']])),
    'fc1' : tf.Variable(tf.random_normal([num_filters['fc1']])),
    'out' : tf.Variable(tf.random_normal([n_classes]))
}

In [86]:
def get_size(ten):
    tens = ten.get_shape();
    val = tens[1];
    for i in range(2, len(tens)):
        val = val * tens[i];
    return val;

def conv2d(x, weights, biases, strides=1):
    x = tf.nn.conv2d(x, weights, strides=[1, strides, strides, 1], padding='SAME');
    x = tf.nn.bias_add(x, biases);
    return tf.nn.relu(x);

def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME');

def conv_net(x, weights, biases, dropout):
    #reshape input picture

    x = tf.reshape(x, shape=[-1, max_size[0], max_size[1], 1]);
    
    #convolution layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'], strides=strides['wc1']);
    print('conv1  ', conv1.get_shape(), get_size(conv1));
    
    conv1 = maxpool2d(conv1, k=weights['mp1']);
    print('pool1 ', conv1.get_shape(), get_size(conv1));

    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'], strides=strides['wc2']);
    print('conv2  ', conv2.get_shape(), get_size(conv2));
    
    conv2 = maxpool2d(conv2, k=weights['mp2']);
    print('pool2  ', conv2.get_shape(), get_size(conv2));
    
    conv3 = conv2d(conv2, weights['wc3'], biases['bc3'], strides=strides['wc3']);
    print('conv3  ', conv3.get_shape(), get_size(conv3));
    
    conv3 = maxpool2d(conv3, k=weights['mp3']);
    print('pool3  ', conv3.get_shape(), get_size(conv3));
    
    fc1 = tf.reshape(conv3, [-1, weights['fc1'].get_shape().as_list()[0]]);
    fc1 = tf.add(tf.matmul(fc1, weights['fc1']), biases['fc1']);
    fc1 = tf.nn.relu(fc1);
    print('fcon1  ', fc1.get_shape(), get_size(fc1));
    
    #output, class prediction
    matrx = tf.matmul(fc1, weights['out']);
    out = tf.add(matrx, biases['out'])
    print('outp1  ', out.get_shape(), get_size(out));
    
    return out;

In [87]:
pred = conv_net(x, weights, biases, keep_prob);

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y));
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost);

correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1));
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32));

init = tf.initialize_all_variables();

conv1   (?, 167, 167, 96) 2677344
pool1  (?, 84, 84, 96) 677376
conv2   (?, 84, 84, 256) 1806336
pool2   (?, 42, 42, 256) 451584
conv3   (?, 42, 42, 384) 677376
pool3   (?, 21, 21, 384) 169344
fcon1   (?, 4096) 4096
outp1   (?, 2) 2


In [88]:
with tf.Session() as session:
    session.run(init);
    batch_length = 5;
    
    for batch_pos in range(0, len(train_images), batch_length):
        batch_x = train_images[batch_pos : batch_pos + batch_length];
        batch_y = train_labels[batch_pos : batch_pos + batch_length];
        #print(batch_x.shape, batch_y.shape);
        session.run(optimizer, feed_dict={x: batch_x, y:batch_y});
        
        if batch_pos % 10 == 0:
            print('completed on batch ', batch_pos);
    
    print('optimization finished');
    
    print('Testing accuracy: ', \
         session.run(accuracy, feed_dict={x: test_images, y:test_labels, keep_prob: 1}));


    session.close();

completed on batch  0
completed on batch  10
completed on batch  20
completed on batch  30
completed on batch  40
completed on batch  50
completed on batch  60
completed on batch  70
completed on batch  80
completed on batch  90
completed on batch  100
completed on batch  110
completed on batch  120
completed on batch  130
completed on batch  140
completed on batch  150
completed on batch  160
completed on batch  170
completed on batch  180
completed on batch  190
completed on batch  200
completed on batch  210
completed on batch  220
completed on batch  230
completed on batch  240
optimization finished
Testing accuracy:  0.568
